In [1]:
import re, functools, timeit

with open("./input/Day-07.txt") as bag_regulations_file:
    bag_regulations_lines = bag_regulations_file.readlines()

# Parse bag regulations to dictionary: outer_bag -> dict([(inner_bag1, amount1), (inner_bag2, amount2), ...])
def create_bag_regulations_dict(bag_regulations_lines):
    bag_regulations_pattern = re.compile("(?P<amount>\d+)? ?(?P<adjective_color>\w+ \w+) bag[s]?\W")
    bag_regulations_parsed = [re.findall(bag_regulations_pattern, bag_regulation) for bag_regulation in bag_regulations_lines]
    return dict(
        (brp[0][1], set((inner_bag[1], int(inner_bag[0])) for inner_bag in brp[1:] if inner_bag[1] != 'no other')) 
         for brp in bag_regulations_parsed)

%timeit create_bag_regulations_dict(bag_regulations_lines)
bag_regulations = create_bag_regulations_dict(bag_regulations_lines)

my_bag_type = 'shiny gold'  # Specifying my own bag type.

5.55 ms ± 255 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Q1: How many bag types contain a shiny gold bag?

In [2]:
def timing1():
    @functools.lru_cache(maxsize=None)
    def contains_bag(outer_bag, given_bag):
        """ Checks if the outer bag contains the given bag (either as inner bag or within one of the inner bags). """
        return (   any(given_bag == inner_bag_spec[0]             for inner_bag_spec in bag_regulations[outer_bag])
                or any(contains_bag(inner_bag_spec[0], given_bag) for inner_bag_spec in bag_regulations[outer_bag]))
    return sum(contains_bag(bag, my_bag_type) for bag in bag_regulations)

%timeit timing1()
print(f"Q1: Bag types containing the {my_bag_type} bag: {timing1()}.")

1.96 ms ± 80.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Q1: Bag types containing the shiny gold bag: 257.


### Q2: How many bags can are inside the shiny gold checked bag?

In [3]:
def timing2():
    @functools.lru_cache(maxsize=None)
    def count_total_checked_bags(checked_bag):
        """ Counts the total number of checked bags (incl. the bag itself). """
        return (1                                                                              # Current bag.
                + sum(inner_bag_details[1] * (count_total_checked_bags(inner_bag_details[0]))  # Inner bags.
                      for inner_bag_details in bag_regulations[checked_bag]))
    return (count_total_checked_bags(my_bag_type) - 1)

%timeit timing2()
print(f"Q2: Number of bags inside the {my_bag_type} bag: {timing2()}.")

34.5 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Q2: Number of bags inside the shiny gold bag: 1038.
